介绍一些tensor的基础操作

In [1]:
import torch
import numpy as np
a=torch.tensor([[1,2],[3,4],[5,6]],dtype=torch.float64, device="cuda:0")
a

tensor([[1., 2.],
        [3., 4.],
        [5., 6.]], device='cuda:0', dtype=torch.float64)

In [2]:
print(a.size())
print(a.shape)


torch.Size([3, 2])
torch.Size([3, 2])


In [3]:
a.numel()#元素总数

6

In [4]:
a.reshape(-1,3)

tensor([[1., 2., 3.],
        [4., 5., 6.]], device='cuda:0', dtype=torch.float64)

In [5]:
zeros=torch.zeros((3,2))
zeros

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

torch.randn((a,b))：生成服从标准正态分布（均值为0，标准差为1）且形状为(a,b)的随机张量   
torch.rand(a, b)：生成服从均匀分布（范围在[0, 1)）且形状为(a,b)的随机数,等价于torch.rand((a, b))

In [6]:
rdm=torch.randn((3,2))
rdu=torch.rand(3,2)
rdm,rdu

(tensor([[ 0.7887, -0.5781],
         [-0.1436,  0.0384],
         [ 0.6465, -0.2272]]),
 tensor([[0.0783, 0.1078],
         [0.0345, 0.8284],
         [0.4919, 0.7628]]))

In [7]:
rdm[0,1]

tensor(-0.5781)

In [8]:
rdm[0,1]=888
rdm

tensor([[ 7.8873e-01,  8.8800e+02],
        [-1.4364e-01,  3.8409e-02],
        [ 6.4649e-01, -2.2717e-01]])

使用赋值符号相当于C++里面的引用，共用同一份数据，使用clone创建副本

In [9]:
ref=rdm[:1,:]
copy=rdm[:1,:].clone()
print(ref)
print(copy)
ref[0,1]=666
copy[0,1]=55

tensor([[7.8873e-01, 8.8800e+02]])
tensor([[7.8873e-01, 8.8800e+02]])


In [10]:
rdm

tensor([[ 7.8873e-01,  6.6600e+02],
        [-1.4364e-01,  3.8409e-02],
        [ 6.4649e-01, -2.2717e-01]])

切片，可以用[-1]选择最后一个元素  
为多个元素赋值相同的值，我们只需要索引所有元素，然后为它们赋值

In [11]:
print(rdm[-1])
rdm[0,:]=10
rdm

tensor([ 0.6465, -0.2272])


tensor([[10.0000, 10.0000],
        [-0.1436,  0.0384],
        [ 0.6465, -0.2272]])

Tensor可以和numpy的ndarray互转

In [12]:
np_rdm=rdm.numpy()
np_rdm

array([[10.        , 10.        ],
       [-0.1436398 ,  0.03840855],
       [ 0.64649487, -0.2271658 ]], dtype=float32)

In [13]:
np_a=np.arange(12).reshape((3,4))
np_a

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

共享内存的互转

In [14]:
ts_a=torch.from_numpy(np_a)#Tensor 与 NumPy 数组共享内存（修改一方会影响另一方）
ts_a

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [15]:
np_ref=rdm.numpy()
ts_ref=torch.as_tensor(np_ref)#接受更多输入类型（如列表、其他 Tensor）
ts_copy=torch.tensor(np_ref)# 复制数据，生成独立 Tensor,不共享内存‌
type(np_ref),type(ts_copy),type(ts_ref)

(numpy.ndarray, torch.Tensor, torch.Tensor)

In [16]:
print(rdm)
rdm[:,1]=5
print(np_ref)
print(ts_ref)
print(ts_copy)

tensor([[10.0000, 10.0000],
        [-0.1436,  0.0384],
        [ 0.6465, -0.2272]])
[[10.          5.        ]
 [-0.1436398   5.        ]
 [ 0.64649487  5.        ]]
tensor([[10.0000,  5.0000],
        [-0.1436,  5.0000],
        [ 0.6465,  5.0000]])
tensor([[10.0000, 10.0000],
        [-0.1436,  0.0384],
        [ 0.6465, -0.2272]])


使用 torchvision.transforms  
将 NumPy 格式的图片（HWC 格式，值域 [0, 255]）转换为 Tensor（CHW 格式，值域 [0.0, 1.0]

In [17]:
from PIL import Image
from torchvision import transforms
 
img = Image.open("image.jpg")  # PIL.Image 对象 （H,W,C）
tensor_trans = transforms.ToTensor()
tensor_img = tensor_trans(img)  # 转为 Tensor (C, H, W)
tensor_img.shape

torch.Size([4, 275, 976])

tensor计算
张量加法有两种形式，一种是直接使用加号，另一种使用torch.add方法

In [18]:
ts_b=torch.randn((3,4))
ts_b

tensor([[-0.8377, -0.7354,  1.1299,  0.5134],
        [-0.4291, -0.7775,  0.7818,  0.8969],
        [-0.8617, -0.2052, -0.8105, -1.1060]])

In [19]:
ts_a+ts_b

tensor([[-0.8377,  0.2646,  3.1299,  3.5134],
        [ 3.5709,  4.2225,  6.7818,  7.8969],
        [ 7.1383,  8.7948,  9.1895,  9.8940]])

In [20]:
torch.add(ts_a,ts_b)

tensor([[-0.8377,  0.2646,  3.1299,  3.5134],
        [ 3.5709,  4.2225,  6.7818,  7.8969],
        [ 7.1383,  8.7948,  9.1895,  9.8940]])

广播broadcast,标量与tensor加减法，会把标量扩张为tensor的维度，然后再逐个元素加减

In [21]:
ts_a+5

tensor([[ 5,  6,  7,  8],
        [ 9, 10, 11, 12],
        [13, 14, 15, 16]])

In [22]:
ts_row=torch.from_numpy(np.arange(2,6).reshape(1,4))
ts_row

tensor([[2, 3, 4, 5]])

矩阵运算，矩阵乘行向量，运算不同于线性代数，这里的计算是针对元素的，要求列数相同，左乘右乘完全相同  
相当于把向量作了广播变成两个形状相同的tensor，再进行计算  
广播机制要求维度相同，标量除外 

In [23]:
print(ts_a.shape)
print(ts_a)
print(ts_row.shape)
print(ts_row)

torch.Size([3, 4])
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
torch.Size([1, 4])
tensor([[2, 3, 4, 5]])


In [24]:
ts_a*ts_row

tensor([[ 0,  3,  8, 15],
        [ 8, 15, 24, 35],
        [16, 27, 40, 55]])

In [25]:
ts_row*ts_a

tensor([[ 0,  3,  8, 15],
        [ 8, 15, 24, 35],
        [16, 27, 40, 55]])

矩阵与列向量，要求行数相同，左乘右乘完全相同  
相当于把向量作了广播变成两个形状相同的tensor，再进行计算

In [26]:
ts_vec=torch.arange(3).reshape(3,1)
ts_vec

tensor([[0],
        [1],
        [2]])

In [27]:
print(ts_a.shape)
print(ts_a)
print(ts_vec.shape)
print(ts_vec)

torch.Size([3, 4])
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
torch.Size([3, 1])
tensor([[0],
        [1],
        [2]])


In [28]:
ts_a*ts_vec

tensor([[ 0,  0,  0,  0],
        [ 4,  5,  6,  7],
        [16, 18, 20, 22]])

In [29]:
ts_vec*ts_a

tensor([[ 0,  0,  0,  0],
        [ 4,  5,  6,  7],
        [16, 18, 20, 22]])

更一般化的广播机制,把两个不同的行列向量分别在长度为1的轴上进行扩展，最终得到两个形状相同的张量，再进行计算

In [30]:
a = torch.arange(3).reshape((3, 1))
b = torch.arange(2).reshape((1, 2))
a, b,a + b


(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]),
 tensor([[0, 1],
         [1, 2],
         [2, 3]]))

矩阵相乘，要求形状相同，左乘右乘完全相同

In [31]:
ts_matrix=torch.from_numpy(np.arange(2,14).reshape(3,4))
ts_matrix

tensor([[ 2,  3,  4,  5],
        [ 6,  7,  8,  9],
        [10, 11, 12, 13]])

In [32]:
print(ts_a.shape)
print(ts_a)
print(ts_matrix.shape)
print(ts_matrix)


torch.Size([3, 4])
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
torch.Size([3, 4])
tensor([[ 2,  3,  4,  5],
        [ 6,  7,  8,  9],
        [10, 11, 12, 13]])


In [33]:
ts_a*ts_matrix

tensor([[  0,   3,   8,  15],
        [ 24,  35,  48,  63],
        [ 80,  99, 120, 143]])

In [34]:
ts_matrix*ts_a

tensor([[  0,   3,   8,  15],
        [ 24,  35,  48,  63],
        [ 80,  99, 120, 143]])

In [35]:
x = torch.tensor([1.0, 2, 4, 8])
y = torch.tensor([2, 2, 2, 2])
x + y, x - y, x * y, x / y, x ** y  # **运算符是求幂运算

(tensor([ 3.,  4.,  6., 10.]),
 tensor([-1.,  0.,  2.,  6.]),
 tensor([ 2.,  4.,  8., 16.]),
 tensor([0.5000, 1.0000, 2.0000, 4.0000]),
 tensor([ 1.,  4., 16., 64.]))

reshape方法，还有一个view方法有一样的效果  
连续张量指的是张量的数据在内存中是连续存储的，没有间隔。也就是说，从第一个元素到最后一个元素，内存地址是连续的，没有跳跃。  
例如，经过转置（transpose(0,1)）的二维张量，其逻辑形状是 [M, N]，但物理存储仍按原始顺序 [N, M] 排列，导致访问时需要跳过其他行的数据，无法直接连续读取  
view()：严格要求输入张量是连续的。若张量非连续（如通过transpose()、permute()等操作后），调用view()会抛出RuntimeError  
reshape()：自动处理非连续张量。若输入非连续，reshape()会隐式复制数据，生成一个新的连续张量，再调整形状，连续输入 → 返回视图（无数据复制）。 

In [36]:
x = torch.randn(6, 8)
y=x.view(-1,12)#负1表示这一个维度通过推断得到。
y

tensor([[-1.1417, -0.7778,  1.9812, -1.8019,  0.6079,  0.1375, -0.7298,  0.8300,
          1.6139, -0.4953, -0.7419,  1.3657],
        [-0.6386, -0.9230, -0.8225,  0.8396, -0.3287, -0.3185, -0.5716, -0.9015,
          1.3355,  0.8460, -1.6919,  0.9251],
        [-0.7419, -0.2791, -0.3919,  1.3135,  1.0622, -1.8698, -2.2066, -1.1838,
         -0.3796,  0.4015,  0.9358, -0.9834],
        [ 0.4111, -1.0796, -0.1684,  1.5222,  2.7072,  0.2290, -0.3648,  0.2585,
         -0.8265, -0.6160,  1.0931, -0.4990]])

In [37]:
z=y.view(48)
z

tensor([-1.1417, -0.7778,  1.9812, -1.8019,  0.6079,  0.1375, -0.7298,  0.8300,
         1.6139, -0.4953, -0.7419,  1.3657, -0.6386, -0.9230, -0.8225,  0.8396,
        -0.3287, -0.3185, -0.5716, -0.9015,  1.3355,  0.8460, -1.6919,  0.9251,
        -0.7419, -0.2791, -0.3919,  1.3135,  1.0622, -1.8698, -2.2066, -1.1838,
        -0.3796,  0.4015,  0.9358, -0.9834,  0.4111, -1.0796, -0.1684,  1.5222,
         2.7072,  0.2290, -0.3648,  0.2585, -0.8265, -0.6160,  1.0931, -0.4990])

In [38]:
y=x.reshape(-1,12)
y

tensor([[-1.1417, -0.7778,  1.9812, -1.8019,  0.6079,  0.1375, -0.7298,  0.8300,
          1.6139, -0.4953, -0.7419,  1.3657],
        [-0.6386, -0.9230, -0.8225,  0.8396, -0.3287, -0.3185, -0.5716, -0.9015,
          1.3355,  0.8460, -1.6919,  0.9251],
        [-0.7419, -0.2791, -0.3919,  1.3135,  1.0622, -1.8698, -2.2066, -1.1838,
         -0.3796,  0.4015,  0.9358, -0.9834],
        [ 0.4111, -1.0796, -0.1684,  1.5222,  2.7072,  0.2290, -0.3648,  0.2585,
         -0.8265, -0.6160,  1.0931, -0.4990]])

In [39]:
z=y.reshape(48)
z

tensor([-1.1417, -0.7778,  1.9812, -1.8019,  0.6079,  0.1375, -0.7298,  0.8300,
         1.6139, -0.4953, -0.7419,  1.3657, -0.6386, -0.9230, -0.8225,  0.8396,
        -0.3287, -0.3185, -0.5716, -0.9015,  1.3355,  0.8460, -1.6919,  0.9251,
        -0.7419, -0.2791, -0.3919,  1.3135,  1.0622, -1.8698, -2.2066, -1.1838,
        -0.3796,  0.4015,  0.9358, -0.9834,  0.4111, -1.0796, -0.1684,  1.5222,
         2.7072,  0.2290, -0.3648,  0.2585, -0.8265, -0.6160,  1.0931, -0.4990])

torch.clamp(data,min,max)  
将data里面的所有数值限定到[min,max]之间，超过边界的等于边界

In [40]:
torch.clamp(z,-0.5,1)

tensor([-0.5000, -0.5000,  1.0000, -0.5000,  0.6079,  0.1375, -0.5000,  0.8300,
         1.0000, -0.4953, -0.5000,  1.0000, -0.5000, -0.5000, -0.5000,  0.8396,
        -0.3287, -0.3185, -0.5000, -0.5000,  1.0000,  0.8460, -0.5000,  0.9251,
        -0.5000, -0.2791, -0.3919,  1.0000,  1.0000, -0.5000, -0.5000, -0.5000,
        -0.3796,  0.4015,  0.9358, -0.5000,  0.4111, -0.5000, -0.1684,  1.0000,
         1.0000,  0.2290, -0.3648,  0.2585, -0.5000, -0.5000,  1.0000, -0.4990])

堆叠cat  
torch.cat(tensors, dim=0, *, out=None):沿指定维度拼接张量序列，不增加新维度，输出张量维度与输入一致  
torch.stack(tensors, dim=0, *, out=None):沿新维度dim堆叠张量序列，输出张量维度+1  
tensors.squeeze():去除冗余维度  
reshape方法返回一个新的张量，但并不会改变原始张量A和B的值，除非赋值给A和B  

以下三种，要求除连接轴外的其他轴形状相同,维度必须相同，不会增加维度  
torch.vstack()：垂直拼接（沿维度0），等价于 torch.cat(..., dim=0)  
torch.hstack():水平拼接（沿维度1），等价于 torch.cat(..., dim=1)  
torch.dstack()：深度堆叠（沿维度2），适用于三维数据（如图像通道叠加）  


In [41]:
A = torch.arange(12).reshape((3, 1,4))
B = torch.arange(8).reshape((2, 1,4))
out=torch.vstack((A,B))
out,out.shape

(tensor([[[ 0,  1,  2,  3]],
 
         [[ 4,  5,  6,  7]],
 
         [[ 8,  9, 10, 11]],
 
         [[ 0,  1,  2,  3]],
 
         [[ 4,  5,  6,  7]]]),
 torch.Size([5, 1, 4]))

In [42]:
out=torch.cat((A,B),0)
out,out.shape

(tensor([[[ 0,  1,  2,  3]],
 
         [[ 4,  5,  6,  7]],
 
         [[ 8,  9, 10, 11]],
 
         [[ 0,  1,  2,  3]],
 
         [[ 4,  5,  6,  7]]]),
 torch.Size([5, 1, 4]))

In [43]:
A=A.reshape(1,3,4)
B=B.reshape(1,2,4)
out=torch.hstack((A,B))
out,out.shape

(tensor([[[ 0,  1,  2,  3],
          [ 4,  5,  6,  7],
          [ 8,  9, 10, 11],
          [ 0,  1,  2,  3],
          [ 4,  5,  6,  7]]]),
 torch.Size([1, 5, 4]))

In [44]:
out=torch.cat((A,B),1)
out,out.shape

(tensor([[[ 0,  1,  2,  3],
          [ 4,  5,  6,  7],
          [ 8,  9, 10, 11],
          [ 0,  1,  2,  3],
          [ 4,  5,  6,  7]]]),
 torch.Size([1, 5, 4]))

In [45]:
A=A.reshape(1,4,3)
B=B.reshape(1,4,2)
out=torch.dstack((A,B))
out,out.shape

(tensor([[[ 0,  1,  2,  0,  1],
          [ 3,  4,  5,  2,  3],
          [ 6,  7,  8,  4,  5],
          [ 9, 10, 11,  6,  7]]]),
 torch.Size([1, 4, 5]))

In [46]:
out=torch.cat((A,B),2)
out,out.shape

(tensor([[[ 0,  1,  2,  0,  1],
          [ 3,  4,  5,  2,  3],
          [ 6,  7,  8,  4,  5],
          [ 9, 10, 11,  6,  7]]]),
 torch.Size([1, 4, 5]))

In [47]:
A = torch.arange(0,12,1).reshape((3, 1,4))
B = torch.arange(5,17,1).reshape((3, 1,4))
A=A.squeeze()
B=B.squeeze()
out=torch.stack((A,B),dim=0)
A.shape,B.shape,out.shape

(torch.Size([3, 4]), torch.Size([3, 4]), torch.Size([2, 3, 4]))

torch.unsqueeze() 在指定维度添加一个长度为1的维度

In [48]:
A=A.unsqueeze(0)
B=torch.unsqueeze(B,dim=0)
A.shape,B.shape

(torch.Size([1, 3, 4]), torch.Size([1, 3, 4]))